In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/



In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small


Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): MIT
 96% 541M/565M [00:01<00:00, 318MB/s]
100% 565M/565M [00:01<00:00, 405MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile('fashion-product-images-small.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/fashion')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence

In [ ]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
model = Sequential([
    model,
    GlobalMaxPooling2D()
])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 2048)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
import numpy as np
from numpy.linalg import norm
import os

def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result


In [ ]:
filenames = []

for file in os.listdir('/content/fashion/images'):
    filenames.append(os.path.join('/content/fashion/images',file))



In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np

class ImageBatchGenerator(Sequence):
    def __init__(self, filenames, batch_size=64, target_size=(224, 224)):
        self.filenames = filenames
        self.batch_size = batch_size
        self.target_size = target_size

    def __len__(self):
        return int(np.ceil(len(self.filenames) / self.batch_size))

    def __getitem__(self, idx):
        batch_files = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = []
        for file in batch_files:
            img = image.load_img(file, target_size=self.target_size)
            img_array = image.img_to_array(img)
            batch_images.append(img_array)
        batch_images = np.array(batch_images)
        batch_images = preprocess_input(batch_images)
        return batch_images, batch_files


In [ ]:
from tqdm import tqdm
from numpy.linalg import norm
import numpy as np

# Create generator
batch_size = 64
generator = ImageBatchGenerator(filenames, batch_size=batch_size)

# Initialize list to store features
all_features = []

# Loop through batches
for i in tqdm(range(len(generator))):
    batch_images, _ = generator[i]
    features = model.predict(batch_images)

    # Normalize each row
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms

    # Append features of this batch
    all_features.append(normalized_features)

# Concatenate all features into a single NumPy array
final_features = np.vstack(all_features)

# Now final_features[i] corresponds to filenames[i]



  0%|          | 0/695 [00:00<?, ?it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step


  0%|          | 1/695 [00:10<2:02:59, 10.63s/it]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


  0%|          | 2/695 [00:11<54:29,  4.72s/it]  

...

100%|█████████▉| 693/695 [05:35<00:00,  2.20it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


100%|█████████▉| 694/695 [05:35<00:00,  2.13it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


100%|██████████| 695/695 [05:42<00:00,  2.03it/s]


In [ ]:
final_features.shape

(44441, 2048)

In [ ]:
import pickle

In [ ]:
pickle.dump(final_features,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))